## Load Libraries and Packages

In [1]:
pwd

u'/Users/2015/Dropbox/Data Science/NYC Data Science Academy/class-projects/Rossmann/Scripts/Python'

In [2]:
import datetime
import numpy as np
from numpy import double
import pandas as pd

## Load Data

In [3]:
print("Loading data start...")
train = pd.read_csv("../../input/train.csv")
test = pd.read_csv("../../input/test.csv")
store = pd.read_csv("../../input/store.csv")
states = pd.read_csv("../../input/store_states.csv")
print("Complete!   : )")

Loading data start...
Complete!   : )


/Users/2015/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Process Data:
#### 1: Impute open = 1 for missing open test dasta
#### 2: Separate date column into year, month, day
#### 3: Convert date column to type 'date' and extract:
* day_of_year
* week_of_year
* quarter
* month start
* month end
* quarter start
* quarter end

#### 4: Drop date and standardize StateHoliday
#### 5: Create promotion one feature
#### 6: Merge data

# 1: Impute Open = 1 for store 622 in test data

In [4]:
test['Open'].fillna(1, inplace=True)
print 'Step 1 Complete.'

Step 1 Complete.


# 2: Create separate Date Columns

In [5]:
train['year'] = train.Date.apply(lambda x: x.split('-')[0])
train['year'] = train['year'].astype(int)
train['month'] = train.Date.apply(lambda x: x.split('-')[1])
train['month'] = train['month'].astype(int)
train['day'] = train.Date.apply(lambda x: x.split('-')[2])
train['day'] = train['day'].astype(int)
print 'Step 2 Complete.'

Step 2 Complete.


In [6]:
test['year'] = test.Date.apply(lambda x: x.split('-')[0])
test['year'] = test['year'].astype(int)
test['month'] = test.Date.apply(lambda x: x.split('-')[1])
test['month'] = test['month'].astype(int)
test['day'] = test.Date.apply(lambda x: x.split('-')[2])
test['day'] = test['day'].astype(int)
print 'Step 2 Complete.'

Step 2 Complete.


In [7]:
train['Date'] = pd.to_datetime(train['Date'])
train['day_of_year'] = train['Date'].dt.dayofyear
train['quarter'] = train['Date'].dt.quarter
train['is_month_start'] = train['Date'].dt.is_month_start.astype(int)
train['is_month_end'] = train['Date'].dt.is_month_end.astype(int)
train['is_quarter_start'] = train['Date'].dt.is_quarter_start.astype(int)
train['is_quarter_end'] = train['Date'].dt.is_quarter_end.astype(int)
print 'Step 3 Complete.'

Step 3 Complete.


In [8]:
# create week_of_year feature since dt.weekofyear doesn't work as expected
lambdafunc = lambda x: pd.Series([x['Date'].isocalendar()[1]])
train[['week_of_year']] = train.apply(lambdafunc, axis=1)

In [9]:
# Check for NaNs
pd.isnull(train).any()

Store               False
DayOfWeek           False
Date                False
Sales               False
Customers           False
Open                False
Promo               False
StateHoliday        False
SchoolHoliday       False
year                False
month               False
day                 False
day_of_year         False
quarter             False
is_month_start      False
is_month_end        False
is_quarter_start    False
is_quarter_end      False
week_of_year        False
dtype: bool

In [11]:
# glance at data frame
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,month,day,day_of_year,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,week_of_year
0,1,5,2015-07-31,5263,555,1,1,0,1,2015,7,31,212,3,0,1,0,0,31
1,2,5,2015-07-31,6064,625,1,1,0,1,2015,7,31,212,3,0,1,0,0,31
2,3,5,2015-07-31,8314,821,1,1,0,1,2015,7,31,212,3,0,1,0,0,31
3,4,5,2015-07-31,13995,1498,1,1,0,1,2015,7,31,212,3,0,1,0,0,31
4,5,5,2015-07-31,4822,559,1,1,0,1,2015,7,31,212,3,0,1,0,0,31


In [12]:
test['Date'] = pd.to_datetime(test['Date'])
test['day_of_year'] = test['Date'].dt.dayofyear
test['quarter'] = test['Date'].dt.quarter
test['is_month_start'] = test['Date'].dt.is_month_start.astype(int)
test['is_month_end'] = test['Date'].dt.is_month_end.astype(int)
test['is_quarter_start'] = test['Date'].dt.is_quarter_start.astype(int)
test['is_quarter_end'] = test['Date'].dt.is_quarter_end.astype(int)
print 'Step 3 Complete.'

Step 3 Complete.


In [13]:
# datetime.date(2015, 7, 31).isocalendar()[1]
lambdafunc = lambda x: pd.Series([x['Date'].isocalendar()[1]])
test[['week_of_year']] = test.apply(lambdafunc, axis=1)

In [14]:
# Check for NaNs
pd.isnull(test).any()

Id                  False
Store               False
DayOfWeek           False
Date                False
Open                False
Promo               False
StateHoliday        False
SchoolHoliday       False
year                False
month               False
day                 False
day_of_year         False
quarter             False
is_month_start      False
is_month_end        False
is_quarter_start    False
is_quarter_end      False
week_of_year        False
dtype: bool

In [15]:
# glance at data frame
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,year,month,day,day_of_year,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,week_of_year
0,1,1,4,2015-09-17,1,1,0,0,2015,9,17,260,3,0,0,0,0,38
1,2,3,4,2015-09-17,1,1,0,0,2015,9,17,260,3,0,0,0,0,38
2,3,7,4,2015-09-17,1,1,0,0,2015,9,17,260,3,0,0,0,0,38
3,4,8,4,2015-09-17,1,1,0,0,2015,9,17,260,3,0,0,0,0,38
4,5,9,4,2015-09-17,1,1,0,0,2015,9,17,260,3,0,0,0,0,38


In [16]:
# 4: Standardize StateHoliday column.
train.ix[(train['StateHoliday'] == 0), 'StateHoliday'] = '0'
print 'Step 4 Complete.'

Step 4 Complete.


In [17]:
# 4: Standardize StateHoliday column.
test.ix[(test['StateHoliday'] == 0), 'StateHoliday'] = '0'
print 'Step 4 Complete.'

Step 4 Complete.


# Create PromoFirstDate and PromoSecondDate Features

In [18]:
train = train.sort(columns = ['Store', 'Date'], ascending = True, na_position = 'last')
train['PromoFirstDate'] = 0
train.ix[((train['Store'] == train['Store'].shift(1)) & (train['Promo'] == 1 ) & (train['Promo'].shift(1) ==0)), 'PromoFirstDate'] = 1
print 'Step 5 Complete.'

Step 5 Complete.


/Users/2015/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [19]:
test = test.sort(columns = ['Store', 'Date'], ascending = True, na_position = 'last')
test['PromoFirstDate'] = 0
test.ix[((test['Store'] == test['Store'].shift(1)) & (test['Promo'] == 1 ) & (test['Promo'].shift(1) ==0)), 'PromoFirstDate'] = 1
print 'Step 5 Complete.'

Step 5 Complete.


/Users/2015/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [20]:
train['PromoSecondDate'] = 0
train.ix[((train['Store'] == train['Store'].shift(2)) & (train['Promo'] == 1 ) & (train['PromoFirstDate'].shift(1) ==1)), 'PromoSecondDate'] = 1

In [21]:
test['PromoSecondDate'] = 0
test.ix[(test['Store'] == test['Store'].shift(2)) & (test['Promo'] == 1 ) & (test['PromoFirstDate'].shift(1) ==1), 'PromoSecondDate'] = 1

# Check how many stores are listed as Open but have 0 sales and/or 0 customers

In [22]:
train.loc[(train['Open'] == 1) & ((train['Sales'] == 0) | (train['Customers'] == 0))].shape

(54, 21)

# Set Open = 0 when Sales = o OR Customers = 0

In [23]:
train.ix[(train['Open'] == 1) & ((train['Sales'] == 0) | (train['Customers'] == 0)), 'Open'] = 0

# Create DayBeforeClosed and DayAfterClosed Features

In [24]:
train['DayBeforeClosed'] = 0
train.ix[((train['Store'] == train['Store'].shift(-1)) & (train['Open'] == 1 ) & (train['Open'].shift(-1) ==0)), 'DayBeforeClosed'] = 1

In [25]:
train['DayAfterClosed'] = 0
train.ix[((train['Store'] == train['Store'].shift(1)) & (train['Open'] == 1 ) & (train['Open'].shift(1) ==0)), 'DayAfterClosed'] = 1

In [26]:
test['DayBeforeClosed'] = 0
test.ix[((test['Store'] == test['Store'].shift(-1)) & (test['Open'] == 1 ) & (test['Open'].shift(-1) ==0)), 'DayBeforeClosed'] = 1

In [27]:
test['DayAfterClosed'] = 0
test.ix[((test['Store'] == test['Store'].shift(1)) & (test['Open'] == 1 ) & (test['Open'].shift(1) ==0)), 'DayAfterClosed'] = 1

In [29]:
# Glance at Data frame
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,...,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,week_of_year,PromoFirstDate,PromoSecondDate,DayBeforeClosed,DayAfterClosed
1016095,1,2,2013-01-01,0,0,0,0,a,1,2013,...,1,1,0,1,0,1,0,0,0,0
1014980,1,3,2013-01-02,5530,668,1,0,0,1,2013,...,1,0,0,0,0,1,0,0,0,1
1013865,1,4,2013-01-03,4327,578,1,0,0,1,2013,...,1,0,0,0,0,1,0,0,0,0
1012750,1,5,2013-01-04,4486,619,1,0,0,1,2013,...,1,0,0,0,0,1,0,0,0,0
1011635,1,6,2013-01-05,4997,635,1,0,0,1,2013,...,1,0,0,0,0,1,0,0,1,0


In [30]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,year,month,...,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,week_of_year,PromoFirstDate,PromoSecondDate,DayBeforeClosed,DayAfterClosed
40232,40233,1,6,2015-08-01,1,0,0,1,2015,8,...,3,1,0,0,0,31,0,0,1,0
39376,39377,1,7,2015-08-02,0,0,0,1,2015,8,...,3,0,0,0,0,31,0,0,0,0
38520,38521,1,1,2015-08-03,1,1,0,1,2015,8,...,3,0,0,0,0,32,1,0,0,1
37664,37665,1,2,2015-08-04,1,1,0,1,2015,8,...,3,0,0,0,0,32,0,1,0,0
36808,36809,1,3,2015-08-05,1,1,0,1,2015,8,...,3,0,0,0,0,32,0,0,0,0


# Create Store Open on Sunday Feature

In [31]:
train['SundayStore'] = 0
train.ix[(train['Open'] == 1) & (train['DayOfWeek'] == 7), 'SundayStore'] = 1

In [32]:
test['SundayStore'] = 0
test.ix[(test['Open'] == 1) & (test['DayOfWeek'] == 7), 'SundayStore'] = 1

# Create Closed Feature

In [33]:
train['Closed'] = (1 - train['Open']).abs();

In [34]:
test['Closed'] = (1 - test['Open']).abs();

# Create Refurbishment Features

In [35]:
train['DayBeforeRefurb'] = 0
train.ix[(
        ((train['Store'] == train['Store'].shift(-1)) 
         & (train['Store']  == train['Store'].shift(-2))
         & (train['Store'] == train['Store'].shift(-3)) 
         & (train['Store'] == train['Store'].shift(-4)) 
         & (train['Store'] == train['Store'].shift(-5))
            ) 
            & ((train['Open'] == 1 ) 
               & (train['Open'].shift(-1) ==0) 
               & (train['Open'].shift(-2) ==0)
               & (train['Open'].shift(-3) ==0)
               & (train['Open'].shift(-4) ==0)
               & (train['Open'].shift(-5) ==0)
              )
    ), 'DayBeforeRefurb'] = 1

In [36]:
test['DayBeforeRefurb'] = 0
test.ix[(
        ((test['Store'] == test['Store'].shift(-1)) 
         & (test['Store']  == test['Store'].shift(-2))
         & (test['Store'] == test['Store'].shift(-3)) 
         & (test['Store'] == test['Store'].shift(-4)) 
         & (test['Store'] == test['Store'].shift(-5))
            ) 
            & ((test['Open'] == 1 ) 
               & (test['Open'].shift(-1) ==0) 
               & (test['Open'].shift(-2) ==0)
               & (test['Open'].shift(-3) ==0)
               & (test['Open'].shift(-4) ==0)
               & (test['Open'].shift(-5) ==0)
              )
    ), 'DayBeforeRefurb'] = 1

In [37]:
train['DayAfterRefurb'] = 0
train.ix[(
        ((train['Store'] == train['Store'].shift(1)) 
         & (train['Store']  == train['Store'].shift(2))
         & (train['Store'] == train['Store'].shift(3)) 
         & (train['Store'] == train['Store'].shift(4)) 
         & (train['Store'] == train['Store'].shift(5))
            ) 
            & ((train['Open'] == 1 ) 
               & (train['Open'].shift(1) ==0) 
               & (train['Open'].shift(2) ==0)
               & (train['Open'].shift(3) ==0)
               & (train['Open'].shift(4) ==0)
               & (train['Open'].shift(5) ==0)
              )
    ), 'DayAfterRefurb'] = 1

In [38]:
test['DayAfterRefurb'] = 0
test.ix[(
        ((test['Store'] == test['Store'].shift(1)) 
         & (test['Store']  == test['Store'].shift(2))
         & (test['Store'] == test['Store'].shift(3)) 
         & (test['Store'] == test['Store'].shift(4)) 
         & (test['Store'] == test['Store'].shift(5))
            ) 
            & ((test['Open'] == 1 ) 
               & (test['Open'].shift(1) ==0) 
               & (test['Open'].shift(2) ==0)
               & (test['Open'].shift(3) ==0)
               & (test['Open'].shift(4) ==0)
               & (test['Open'].shift(5) ==0)
              )
    ), 'DayAfterRefurb'] = 1

In [39]:
train['DaysBeforeRefurb'] = 0
train.ix[(train['DayBeforeRefurb'].shift(0) ==1), 'DaysBeforeRefurb'] = 1
train.ix[(train['DayBeforeRefurb'].shift(-1) ==1), 'DaysBeforeRefurb'] = 2
train.ix[(train['DayBeforeRefurb'].shift(-2) ==1), 'DaysBeforeRefurb'] = 3
train.ix[(train['DayBeforeRefurb'].shift(-3) ==1), 'DaysBeforeRefurb'] = 4
train.ix[(train['DayBeforeRefurb'].shift(-4) ==1), 'DaysBeforeRefurb'] = 5
train.ix[(train['DayBeforeRefurb'].shift(-5) ==1), 'DaysBeforeRefurb'] = 6
train.ix[(train['DayBeforeRefurb'].shift(-6) ==1), 'DaysBeforeRefurb'] = 7
train.ix[(train['DayBeforeRefurb'].shift(-7) ==1), 'DaysBeforeRefurb'] = 8
train.ix[(train['DayBeforeRefurb'].shift(-8) ==1), 'DaysBeforeRefurb'] = 9
train.ix[(train['DayBeforeRefurb'].shift(-9) ==1), 'DaysBeforeRefurb'] = 10
train.ix[(train['DayBeforeRefurb'].shift(-10) ==1), 'DaysBeforeRefurb'] = 11
train.ix[(train['DayBeforeRefurb'].shift(-11) ==1), 'DaysBeforeRefurb'] = 12
train.ix[(train['DayBeforeRefurb'].shift(-12) ==1), 'DaysBeforeRefurb'] = 13
train.ix[(train['DayBeforeRefurb'].shift(-13) ==1), 'DaysBeforeRefurb'] = 14
train.ix[(train['DayBeforeRefurb'].shift(-14) ==1), 'DaysBeforeRefurb'] = 15

In [40]:
test['DaysBeforeRefurb'] = 0
test.ix[(test['DayBeforeRefurb'].shift(0) ==1), 'DaysBeforeRefurb'] = 1
test.ix[(test['DayBeforeRefurb'].shift(-1) ==1), 'DaysBeforeRefurb'] = 2
test.ix[(test['DayBeforeRefurb'].shift(-2) ==1), 'DaysBeforeRefurb'] = 3
test.ix[(test['DayBeforeRefurb'].shift(-3) ==1), 'DaysBeforeRefurb'] = 4
test.ix[(test['DayBeforeRefurb'].shift(-4) ==1), 'DaysBeforeRefurb'] = 5
test.ix[(test['DayBeforeRefurb'].shift(-5) ==1), 'DaysBeforeRefurb'] = 6
test.ix[(test['DayBeforeRefurb'].shift(-6) ==1), 'DaysBeforeRefurb'] = 7
test.ix[(test['DayBeforeRefurb'].shift(-7) ==1), 'DaysBeforeRefurb'] = 8
test.ix[(test['DayBeforeRefurb'].shift(-8) ==1), 'DaysBeforeRefurb'] = 9
test.ix[(test['DayBeforeRefurb'].shift(-9) ==1), 'DaysBeforeRefurb'] = 10
test.ix[(test['DayBeforeRefurb'].shift(-10) ==1), 'DaysBeforeRefurb'] = 11
test.ix[(test['DayBeforeRefurb'].shift(-11) ==1), 'DaysBeforeRefurb'] = 12
test.ix[(test['DayBeforeRefurb'].shift(-12) ==1), 'DaysBeforeRefurb'] = 13
test.ix[(test['DayBeforeRefurb'].shift(-13) ==1), 'DaysBeforeRefurb'] = 14
test.ix[(test['DayBeforeRefurb'].shift(-14) ==1), 'DaysBeforeRefurb'] = 15

In [41]:
train['DaysAfterRefurb'] = 0
train.ix[(train['DayAfterRefurb'].shift(0) ==1), 'DaysAfterRefurb'] = 1
train.ix[(train['DayAfterRefurb'].shift(1) ==1), 'DaysAfterRefurb'] = 2
train.ix[(train['DayAfterRefurb'].shift(2) ==1), 'DaysAfterRefurb'] = 3
train.ix[(train['DayAfterRefurb'].shift(3) ==1), 'DaysAfterRefurb'] = 4
train.ix[(train['DayAfterRefurb'].shift(4) ==1), 'DaysAfterRefurb'] = 5
train.ix[(train['DayAfterRefurb'].shift(5) ==1), 'DaysAfterRefurb'] = 6
train.ix[(train['DayAfterRefurb'].shift(6) ==1), 'DaysAfterRefurb'] = 7
train.ix[(train['DayAfterRefurb'].shift(7) ==1), 'DaysAfterRefurb'] = 8
train.ix[(train['DayAfterRefurb'].shift(8) ==1), 'DaysAfterRefurb'] = 9
train.ix[(train['DayAfterRefurb'].shift(9) ==1), 'DaysAfterRefurb'] = 10
train.ix[(train['DayAfterRefurb'].shift(10) ==1), 'DaysAfterRefurb'] = 11
train.ix[(train['DayAfterRefurb'].shift(11) ==1), 'DaysAfterRefurb'] = 12
train.ix[(train['DayAfterRefurb'].shift(12) ==1), 'DaysAfterRefurb'] = 13
train.ix[(train['DayAfterRefurb'].shift(13) ==1), 'DaysAfterRefurb'] = 14
train.ix[(train['DayAfterRefurb'].shift(14) ==1), 'DaysAfterRefurb'] = 15

In [42]:
test['DaysAfterRefurb'] = 0
test.ix[(test['DayAfterRefurb'].shift(0) ==1), 'DaysAfterRefurb'] = 1
test.ix[(test['DayAfterRefurb'].shift(1) ==1), 'DaysAfterRefurb'] = 2
test.ix[(test['DayAfterRefurb'].shift(2) ==1), 'DaysAfterRefurb'] = 3
test.ix[(test['DayAfterRefurb'].shift(3) ==1), 'DaysAfterRefurb'] = 4
test.ix[(test['DayAfterRefurb'].shift(4) ==1), 'DaysAfterRefurb'] = 5
test.ix[(test['DayAfterRefurb'].shift(5) ==1), 'DaysAfterRefurb'] = 6
test.ix[(test['DayAfterRefurb'].shift(6) ==1), 'DaysAfterRefurb'] = 7
test.ix[(test['DayAfterRefurb'].shift(7) ==1), 'DaysAfterRefurb'] = 8
test.ix[(test['DayAfterRefurb'].shift(8) ==1), 'DaysAfterRefurb'] = 9
test.ix[(test['DayAfterRefurb'].shift(9) ==1), 'DaysAfterRefurb'] = 10
test.ix[(test['DayAfterRefurb'].shift(10) ==1), 'DaysAfterRefurb'] = 11
test.ix[(test['DayAfterRefurb'].shift(11) ==1), 'DaysAfterRefurb'] = 12
test.ix[(test['DayAfterRefurb'].shift(12) ==1), 'DaysAfterRefurb'] = 13
test.ix[(test['DayAfterRefurb'].shift(13) ==1), 'DaysAfterRefurb'] = 14
test.ix[(test['DayAfterRefurb'].shift(14) ==1), 'DaysAfterRefurb'] = 15

In [43]:
train.head(25)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,...,PromoFirstDate,PromoSecondDate,DayBeforeClosed,DayAfterClosed,SundayStore,Closed,DayBeforeRefurb,DayAfterRefurb,DaysBeforeRefurb,DaysAfterRefurb
1016095,1,2,2013-01-01,0,0,0,0,a,1,2013,...,0,0,0,0,0,1,0,0,0,0
1014980,1,3,2013-01-02,5530,668,1,0,0,1,2013,...,0,0,0,1,0,0,0,0,0,0
1013865,1,4,2013-01-03,4327,578,1,0,0,1,2013,...,0,0,0,0,0,0,0,0,0,0
1012750,1,5,2013-01-04,4486,619,1,0,0,1,2013,...,0,0,0,0,0,0,0,0,0,0
1011635,1,6,2013-01-05,4997,635,1,0,0,1,2013,...,0,0,1,0,0,0,0,0,0,0
1010520,1,7,2013-01-06,0,0,0,0,0,1,2013,...,0,0,0,0,0,1,0,0,0,0
1009405,1,1,2013-01-07,7176,785,1,1,0,1,2013,...,1,0,0,1,0,0,0,0,0,0
1008290,1,2,2013-01-08,5580,654,1,1,0,1,2013,...,0,1,0,0,0,0,0,0,0,0
1007175,1,3,2013-01-09,5471,626,1,1,0,1,2013,...,0,0,0,0,0,0,0,0,0,0
1006060,1,4,2013-01-10,4892,615,1,1,0,1,2013,...,0,0,0,0,0,0,0,0,0,0


In [44]:
test.head(25)

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,year,month,...,PromoFirstDate,PromoSecondDate,DayBeforeClosed,DayAfterClosed,SundayStore,Closed,DayBeforeRefurb,DayAfterRefurb,DaysBeforeRefurb,DaysAfterRefurb
40232,40233,1,6,2015-08-01,1,0,0,1,2015,8,...,0,0,1,0,0,0,0,0,0,0
39376,39377,1,7,2015-08-02,0,0,0,1,2015,8,...,0,0,0,0,0,1,0,0,0,0
38520,38521,1,1,2015-08-03,1,1,0,1,2015,8,...,1,0,0,1,0,0,0,0,0,0
37664,37665,1,2,2015-08-04,1,1,0,1,2015,8,...,0,1,0,0,0,0,0,0,0,0
36808,36809,1,3,2015-08-05,1,1,0,1,2015,8,...,0,0,0,0,0,0,0,0,0,0
35952,35953,1,4,2015-08-06,1,1,0,1,2015,8,...,0,0,0,0,0,0,0,0,0,0
35096,35097,1,5,2015-08-07,1,1,0,1,2015,8,...,0,0,0,0,0,0,0,0,0,0
34240,34241,1,6,2015-08-08,1,0,0,1,2015,8,...,0,0,1,0,0,0,0,0,0,0
33384,33385,1,7,2015-08-09,0,0,0,1,2015,8,...,0,0,0,0,0,1,0,0,0,0
32528,32529,1,1,2015-08-10,1,0,0,1,2015,8,...,0,0,0,1,0,0,0,0,0,0


# Glance at Store 909 to see if Refurbishment Features were created correctly

In [46]:
# Store being refurbished between 2015-07-02 and 2015-07-18
train.loc[(train['Store'] == 909) &  (train['year'] == 2015) & (train['month'] == 6)]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,...,PromoFirstDate,PromoSecondDate,DayBeforeClosed,DayAfterClosed,SundayStore,Closed,DayBeforeRefurb,DayAfterRefurb,DaysBeforeRefurb,DaysAfterRefurb
67808,909,1,2015-06-01,15844,1279,1,1,0,1,2015,...,1,0,0,1,0,0,0,0,0,0
66693,909,2,2015-06-02,14629,1234,1,1,0,1,2015,...,0,1,0,0,0,0,0,0,0,0
65578,909,3,2015-06-03,13209,1176,1,1,0,1,2015,...,0,0,1,0,0,0,0,0,0,0
64463,909,4,2015-06-04,0,0,0,1,a,1,2015,...,0,0,0,0,0,1,0,0,0,0
63348,909,5,2015-06-05,15352,1260,1,1,0,1,2015,...,0,0,0,1,0,0,0,0,0,0
62233,909,6,2015-06-06,8868,930,1,0,0,0,2015,...,0,0,1,0,0,0,0,0,0,0
61118,909,7,2015-06-07,0,0,0,0,0,0,2015,...,0,0,0,0,0,1,0,0,0,0
60003,909,1,2015-06-08,9352,978,1,0,0,0,2015,...,0,0,0,1,0,0,0,0,0,0
58888,909,2,2015-06-09,9778,1006,1,0,0,0,2015,...,0,0,0,0,0,0,0,0,0,0
57773,909,3,2015-06-10,8885,933,1,0,0,0,2015,...,0,0,0,0,0,0,0,0,0,0


In [47]:
# Store being refurbished between 2015-07-02 and 2015-07-18
train.loc[(train['Store'] == 909) &  (train['year'] == 2015) & (train['month'] == 7)]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,...,PromoFirstDate,PromoSecondDate,DayBeforeClosed,DayAfterClosed,SundayStore,Closed,DayBeforeRefurb,DayAfterRefurb,DaysBeforeRefurb,DaysAfterRefurb
34358,909,3,2015-07-01,3547,438,1,1,0,0,2015,...,0,0,1,0,0,0,1,0,1,0
33243,909,4,2015-07-02,0,0,0,1,0,0,2015,...,0,0,0,0,0,1,0,0,0,0
32128,909,5,2015-07-03,0,0,0,1,0,0,2015,...,0,0,0,0,0,1,0,0,0,0
31013,909,6,2015-07-04,0,0,0,0,0,0,2015,...,0,0,0,0,0,1,0,0,0,0
29898,909,7,2015-07-05,0,0,0,0,0,0,2015,...,0,0,0,0,0,1,0,0,0,0
28783,909,1,2015-07-06,0,0,0,0,0,0,2015,...,0,0,0,0,0,1,0,0,0,0
27668,909,2,2015-07-07,0,0,0,0,0,0,2015,...,0,0,0,0,0,1,0,0,0,0
26553,909,3,2015-07-08,0,0,0,0,0,0,2015,...,0,0,0,0,0,1,0,0,0,0
25438,909,4,2015-07-09,0,0,0,0,0,0,2015,...,0,0,0,0,0,1,0,0,0,0
24323,909,5,2015-07-10,0,0,0,0,0,0,2015,...,0,0,0,0,0,1,0,0,0,0


# Join with States Data Frame

In [48]:
states.head()

,Store,State
0,1,HE
1,2,TH
2,3,NW
3,4,BE
4,5,SN


In [49]:
train.shape

(1017209, 29)

In [50]:
train_states = pd.merge(train, states, on = 'Store', how = 'inner')

In [51]:
train_states.shape

(1017209, 30)

In [53]:
train_states.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,...,PromoSecondDate,DayBeforeClosed,DayAfterClosed,SundayStore,Closed,DayBeforeRefurb,DayAfterRefurb,DaysBeforeRefurb,DaysAfterRefurb,State
0,1,2,2013-01-01,0,0,0,0,a,1,2013,...,0,0,0,0,1,0,0,0,0,HE
1,1,3,2013-01-02,5530,668,1,0,0,1,2013,...,0,0,1,0,0,0,0,0,0,HE
2,1,4,2013-01-03,4327,578,1,0,0,1,2013,...,0,0,0,0,0,0,0,0,0,HE
3,1,5,2013-01-04,4486,619,1,0,0,1,2013,...,0,0,0,0,0,0,0,0,0,HE
4,1,6,2013-01-05,4997,635,1,0,0,1,2013,...,0,1,0,0,0,0,0,0,0,HE


In [54]:
pd.isnull(train_states).any()

Store               False
DayOfWeek           False
Date                False
Sales               False
Customers           False
Open                False
Promo               False
StateHoliday        False
SchoolHoliday       False
year                False
month               False
day                 False
day_of_year         False
quarter             False
is_month_start      False
is_month_end        False
is_quarter_start    False
is_quarter_end      False
week_of_year        False
PromoFirstDate      False
PromoSecondDate     False
DayBeforeClosed     False
DayAfterClosed      False
SundayStore         False
Closed              False
DayBeforeRefurb     False
DayAfterRefurb      False
DaysBeforeRefurb    False
DaysAfterRefurb     False
State               False
dtype: bool

In [55]:
def rstr(df): return df.shape, df.apply(lambda x: [x.unique()])
print(rstr(train_states))

((1017209, 30), Store               [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1...
DayOfWeek                                     [[2, 3, 4, 5, 6, 7, 1]]
Date                [[2013-01-01 00:00:00, 2013-01-02 00:00:00, 20...
Sales               [[0, 5530, 4327, 4486, 4997, 7176, 5580, 5471,...
Customers           [[0, 668, 578, 619, 635, 785, 654, 626, 615, 5...
Open                                                         [[0, 1]]
Promo                                                        [[0, 1]]
StateHoliday                                           [[a, 0, b, c]]
SchoolHoliday                                                [[1, 0]]
year                                             [[2013, 2014, 2015]]
month                       [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]
day                 [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1...
day_of_year         [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1...
quarter                                                [[1, 2, 3, 4]]
is_m

In [56]:
print(rstr(test))

((41088, 28), Id                  [[40233, 39377, 38521, 37665, 36809, 35953, 35...
Store               [[1, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1...
DayOfWeek                                     [[6, 7, 1, 2, 3, 4, 5]]
Date                [[2015-08-01 00:00:00, 2015-08-02 00:00:00, 20...
Open                                                     [[1.0, 0.0]]
Promo                                                        [[0, 1]]
StateHoliday                                                 [[0, a]]
SchoolHoliday                                                [[1, 0]]
year                                                         [[2015]]
month                                                        [[8, 9]]
day                 [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1...
day_of_year         [[213, 214, 215, 216, 217, 218, 219, 220, 221,...
quarter                                                         [[3]]
is_month_start                                               [[1, 0]]
is_mon

In [57]:
test.shape

(41088, 28)

In [58]:
test_states = pd.merge(test, states, on = 'Store', how = 'inner')
print(rstr(test_states))

((41088, 29), Id                  [[40233, 39377, 38521, 37665, 36809, 35953, 35...
Store               [[1, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1...
DayOfWeek                                     [[6, 7, 1, 2, 3, 4, 5]]
Date                [[2015-08-01 00:00:00, 2015-08-02 00:00:00, 20...
Open                                                     [[1.0, 0.0]]
Promo                                                        [[0, 1]]
StateHoliday                                                 [[0, a]]
SchoolHoliday                                                [[1, 0]]
year                                                         [[2015]]
month                                                        [[8, 9]]
day                 [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1...
day_of_year         [[213, 214, 215, 216, 217, 218, 219, 220, 221,...
quarter                                                         [[3]]
is_month_start                                               [[1, 0]]
is_mon

In [59]:
pd.isnull(test_states).any()

Id                  False
Store               False
DayOfWeek           False
Date                False
Open                False
Promo               False
StateHoliday        False
SchoolHoliday       False
year                False
month               False
day                 False
day_of_year         False
quarter             False
is_month_start      False
is_month_end        False
is_quarter_start    False
is_quarter_end      False
week_of_year        False
PromoFirstDate      False
PromoSecondDate     False
DayBeforeClosed     False
DayAfterClosed      False
SundayStore         False
Closed              False
DayBeforeRefurb     False
DayAfterRefurb      False
DaysBeforeRefurb    False
DaysAfterRefurb     False
State               False
dtype: bool

In [60]:
test_states.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,year,month,...,PromoSecondDate,DayBeforeClosed,DayAfterClosed,SundayStore,Closed,DayBeforeRefurb,DayAfterRefurb,DaysBeforeRefurb,DaysAfterRefurb,State
0,40233,1,6,2015-08-01,1,0,0,1,2015,8,...,0,1,0,0,0,0,0,0,0,HE
1,39377,1,7,2015-08-02,0,0,0,1,2015,8,...,0,0,0,0,1,0,0,0,0,HE
2,38521,1,1,2015-08-03,1,1,0,1,2015,8,...,0,0,1,0,0,0,0,0,0,HE
3,37665,1,2,2015-08-04,1,1,0,1,2015,8,...,1,0,0,0,0,0,0,0,0,HE
4,36809,1,3,2015-08-05,1,1,0,1,2015,8,...,0,0,0,0,0,0,0,0,0,HE


# Output CSV files for R

In [61]:
train_states.to_csv("../../KaggleProject/data/train_states_R_v5.csv", index=False)

In [ ]:
test_states.to_csv("../../KaggleProject/data/test_states_R_v5.csv", index=False)